In [1]:
from sessions import *
from venueordering import *
from pylatex_textboxes import *
from formatting_solver import *

In [2]:
import itertools as it
import os
import requests
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient

def connect_api_sess():
    client_secret = os.environ["GREN_CLIENT_SECRET"]
    assert len(client_secret)>0
    client_id = os.environ["GREN_CLIENT_ID"]
    base_url = os.environ["GREN_BASE_URL"]
    event="swancon2017"

    client = BackendApplicationClient(client_id=client_id)
    oath_sess = OAuth2Session(client=client)
    token = oath_sess.fetch_token(token_url=base_url+'oauth/token', client_id=client_id, client_secret=client_secret)
    return oath_sess, base_url, event


In [3]:
def load_grenadine_session(prog_item):
    id = prog_item['id']
    start = prog_item['start_time']
    end = prog_item['end_time']
    title = prog_item['title']
    tags = [entry['name'] for entry in prog_item['base_tags']]
    try:
        tags.append(prog_item['format']['name'])
    except KeyError:
        pass
    end
    people = [entry['person']['full_publication_name'] for entry in prog_item['sorted_published_item_assignments']]
    venues = [prog_item['published_room']['name']]
    description = prog_item['description']
    
    return session(id, start, end, title, tags, people, venues, description)


def fetch_sessions():
    api_sess, base_url, event = connect_api_sess() 
    

    url = base_url+event+"/planner_apis/1/published_items" 
    result =  api_sess.get(url, params={'paginated':'false'})
    raw_sessions = result.json()['response']
    return list(map(load_grenadine_session, raw_sessions))


In [4]:
def replace_room_with_rooms(sessions, old, *news):
    for sess in sessions:
        while old in sess.venues:
            print(sess.title)
            sess.venues.remove(old)
            sess.venues.extend(news)
    
    sessions
    

"Find multiple copies of events for different rooms at the same time, and convert them to single events in spanning rooms"
def bridge_rooms(sessions):
    bridged_sessions = []
    
    sessions.sort(key=lambda x: (x.start, x.end, x.title))
    for title, title_matchs in it.groupby(sessions, lambda x: x.title):
        title_matchs = list(title_matchs) #GoldPlate: this doesn't have to be done
        head = title_matchs[0]
        tail = title_matchs[1:]
        for sess in tail:
            head.venues.extend(sess.venues)
        bridged_sessions.append(head)
    
    sessions[:]=bridged_sessions[:]
    return sessions


In [5]:
sessions = fetch_sessions()
replace_room_with_rooms(sessions, 'Swan Rooms', 'Swan Room, Black', 'Swan Room, White')
bridge_rooms(sessions);

Dinner
Hitchhiker's Guide to Swancon (Opening Ceremony)
Meet The AV Crew
Douglas Adams: A life remembered
Lunch
Dinner
Lunch
2018 Launch Set up
Swancon 2018 Launch
Dinner
The Restaurant At The End of The Universe (Masquerade Ball)
Lunch
Dinner
Don't judge a book by it's cover
Lunch
Room Set up Closing Ceremony
So Long and Thanks for All The Fish (Closing Ceremony)


In [6]:
def fetch_venue_orderer():
    api_sess, base_url, event = connect_api_sess() 
    url = base_url+event+"/planner_apis/1/published_rooms" 
    result =  api_sess.get(url, params={'paginated':'false'})
    raw_rooms = result.json()['response']
    fetched_venues = [vv['name'] for vv in raw_rooms]
    
    def orderer(sessions):
        used_venues = set.union(*(set(sess.venues) for sess in sessions))
        return [vn for vn in fetched_venues if vn in used_venues]
        
    return orderer

In [7]:

timetable_metric_solver(sessions,1.7,2.15,'cm', 1.7, fetch_venue_orderer())

In [8]:
from pylatex.utils import escape_latex, NoEscape
from pylatex.utils import NoEscape
from pagelayout import Multicols
from itertools import groupby
from pylatex.base_classes import Environment

class Minipage(Environment):
    def __init__(self, width):
        Environment.__init__(self,arguments=[width])


def write_descriptions(sessions, doc):
    for day_name, day_session in groupby(sessions, lambda ss: ss.day):
        #with doc.create(Section(day_name,numbering=False)):
            with doc.create(Multicols(2)):
                doc.append(NoEscape("[\section*{%s}]" %day_name))
                for session in day_session:
                    if len(session.description)==0:
                        continue
                    title = session.title
                    if "AdultsOnly" in session.tags:
                        title+=NoEscape(" [ADULT]")
                    with doc.create(Subsection(title,numbering=False)):
                        with doc.create(Description()) as desc:
                            doc.append(Command("setlength",[NoEscape("\itemsep"),"0pt"]))
                            doc.append(Command("setlength",[NoEscape("\parsep"),"0pt"]))
                            doc.append(Command("setlength",[NoEscape("\parskip"),"0pt"]))
                            desc.add_item("When:", session.day+", "+session.start_time+" -- "+session.end_time)

                            if len(session.venues)>0:
                                desc.add_item("Where:", ", ".join(session.venues))
                            if len(session.people)>0:
                                desc.add_item("Who:", ", ".join(session.people))
                            #if len(session.tags)>0:
                            #    desc.add_item("Tags:", ", ".join(session.tags))
                        doc.append(session.description)
                #doc.append(Command("newpage"))

In [9]:
import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package, lists
from pylatex.lists import Description

from pylatex.utils import escape_latex, NoEscape
from itertools import groupby
from pylatex.base_classes.command import Options
from pylatex.utils import escape_latex

def write_venues(doc, tt_solver):
    doc.append(textpos_origin('2cm','2cm'))
    doc.append(TextcolorboxStyle('sharp corners','fit algorithm=hybrid*','center upper', valign='center',
                                 colframe='blue!50!black',colback='blue!10!white',
                                 boxsep='0pt',top='0mm',bottom='0mm',left='0mm',right='0mm'))
            
    for venue in tt_solver.venues:
        doc.append(FixedTextbox(venue,
                                tt_solver.get_venue_x(venue),
                                '0cm', 
                                '1.5cm',
                                tt_solver.get_venue_width())
                               )

def make_pretty_timetable(doc,sessions, venue_orderer):
    tt_solver = timetable_metric_solver(sessions,1.7,2.15,'cm', 1.7, venue_orderer)

    
    for date, day_sessions in groupby(sessions, lambda ss: ss.start.date()):   
        day_str = str(DAYS[date.weekday()])
        with doc.create(Subsection(NoEscape(day_str+" \hfill "+day_str+" \hfill "+day_str), numbering=False)):
           
            write_venues(doc,tt_solver)
            doc.append(TextcolorboxStyle('fit algorithm=hybrid*','rounded corners', 'center upper', valign='center',
                                 colframe='blue!50!black',colback='white!10!white',
                                 boxsep='1pt',top='0mm',bottom='0mm',left='0mm',right='0mm'))
            
            day_sessions = sorted(day_sessions, key = lambda ss: -len(ss.venues) or -len(tt_solver.venues)-1)
            for session in day_sessions:
                #print("*", session.title)
                colback = tt_solver.get_color(session)
                tcb_options = Options(colback=colback) if colback else None
                doc.append(FixedTextbox(NoEscape('%s \\\\ \\tcbfontsize{0.75} %s -- %s ' % 
                                                     tuple(map(escape_latex, (session.title, session.start_time, session.end_time)))),
                                        tt_solver.get_x(session),
                                        tt_solver.get_y(session), 
                                        tt_solver.get_height(session),
                                        tt_solver.get_width(session),
                                        tcb_options=tcb_options))

            doc.append(Command('newpage'))



In [13]:
datetime.now()

datetime.datetime(2017, 4, 3, 12, 20, 12, 84475)

In [14]:




doc = Document(documentclass="scrreprt")
margins=['tmargin=1.5cm','bmargin=2.5cm','lmargin=2.5cm','rmargin=2.5cm',]
doc.packages.append(Package('geometry', options=margins))
doc.packages.append(Package('xcolor', options=["svgnames","dvipsnames"]))

doc.packages.append(Package("draftwatermark"))
doc.append(Command("newgeometry",arguments=",".join(margins)))
doc.append(Command("SetWatermarkText", "Draft v0.1.0.print"))
doc.append(Command("SetWatermarkScale", "0.5"))
doc.append(Command("SetWatermarkColor", "0.9,0.59,0.5", "rgb"))

make_pretty_timetable(doc,sessions, fetch_venue_orderer())
write_descriptions(sessions,doc)

from IPython.display import FileLink, FileLinks
with open("out/exported.tex", 'w') as fh:
    doc.dump(fh)

!lualatex --output-directory=out --interaction=nonstopmode out/exported.tex
FileLinks("./out")
FileLinks("./out")

This is LuaTeX, Version beta-0.79.1 (TeX Live 2015/dev/Debian) (rev 4971) 
 restricted \write18 enabled.
(./out/exported.tex
LaTeX2e <2014/05/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrreprt.cls
Document Class: scrreprt 2013/12/19 v3.12 KOMA-Script document class (report)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrkbase.sty
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrbase.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrlfile.sty
Package scrlfile, 2013/12/19 v3.12 KOMA-Script package (loading files)
                  Copyright (C) Markus Kohm

))) (/usr/share/texlive/texmf-dist/tex/latex/koma-script/tocbasic.sty)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/scrsize11pt.clo)
(/usr/share/texlive/texmf-dist/tex/latex/koma-script/typearea.sty
Package typearea, 2013/12/19 v3.12 KOMA-Script package (type area

./out/
  exported.pdf
  exported.log
  exported.tex
  exported.aux

In [ ]:
#ROOMs/DAYS

import pylatex
from pylatex import Document, Section, Subsection, Subsubsection, Table, Package,lists
from pylatex.lists import Description
from pylatex.base_classes import Command


from itertools import groupby

from collections import defaultdict
vds = defaultdict(lambda : defaultdict(list))
for date, day_sessions in groupby(sessions, lambda ss: ss.start.date()):
    for sess in day_sessions:
        for venue in sess.venues:
            vds[venue][date].append(sess)

#################################
doc = Document(documentclass="standalone")
#doc.packages.add("enumitem")
#doc.append(r"\setlist[description]{leftmargin=10em,labelindent=0em")
for venue in vds.keys():
    sessions_on_page=0
    doc.append(Command("newpage"))
    for date in sorted(vds[venue]):
        day = DAYS[date.weekday()]
        with doc.create(Section(NoEscape("%s\\\\ %s" % (day,venue.replace("(","\\\\("))), numbering=False)):
            doc.append(Command("vfill"))
            with doc.create(Description()) as sched:
                sesses = sorted(vds[venue][date], key=lambda ss: ss.start)
                sessions_on_page+=len(sesses)
                for sess in sesses:
                    sched.add_item(NoEscape("%s -- %s" % (sess.start_time, sess.end_time)), sess.title)
            doc.append(Command("vfill"))
            if sessions_on_page>=6:
                doc.append(Command("newpage"))
                sessions_on_page=0
            else:
                doc.append(Command("vfill"))
                

In [ ]:
from IPython.display import FileLink, FileLinks
with open("out/exported_days.tex", 'w') as temp_out:
    doc.dump(temp_out)

#!lualatex --output-directory=out --interaction=nonstopmode exported_days.tex
FileLinks("./out")
